In [1]:
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.semantic_kernel import SKChatCompletionAdapter
from autogen_core.models import ModelFamily, UserMessage
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.anthropic import AnthropicChatCompletion, AnthropicChatPromptExecutionSettings
from semantic_kernel.memory.null_memory import NullMemory

import re
import os
from dotenv import load_dotenv

load_dotenv('../../.env')

True

In [2]:
import sys
import importlib
sys.path.append('../..')
from server.custom_types import MCT_Node
import criteria

/Users/shichengwen/miniconda3/envs/task-decomp/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/shichengwen/miniconda3/envs/task-decomp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
complexity_few_shot_examples = [
    {
        "user_evaluation": 2,
        "user_reasoning": "This step is crucial as it allows you to identify the key components (entities) and how they are related to each other, which is foundational for constructing a knowledge graph.",
        "node": "This step is crucial as it allows you to identify the key components (entities) and how they are related to each other, which is foundational for constructing a knowledge graph.",
    },
    {
        "user_evaluation": 0,
        "user_reasoning": "This step is not crucial as it does not allow you to identify the key components (entities) and how they are related to each other, which is foundational for constructing a knowledge graph.",
        "node": "This step is not crucial as it does not allow you to identify the key components (entities) and how they are related to each other, which is foundational for constructing a knowledge graph.",
    },
    {
        "user_evaluation": 1,
        "user_reasoning": "This step is not crucial as it does not allow you to identify the key components (entities) and how they are related to each other, which is foundational for constructing a knowledge graph.",
        "node": "This step is not crucial as it does not allow you to identify the key components (entities) and how they are related to each other, which is foundational for constructing a knowledge graph.",
    },
]

In [6]:
importlib.reload(criteria)

complexity_definition = (
    "A text is considered complex if it requires advanced knowledge "
    "or expertise, contains multiple layered or specialized concepts, or "
    "requires multi-step reasoning to understand or accomplish the described goal. "
    "Otherwise, it's considered not complex."
)

coherence_definition = (
    "Two text pieces are considered coherent in a sequence if the second "
    "logically or thematically follows from the first, maintains consistency with it, "
    "and does not present a contradictory or unrelated concept."
)

importance_definition = (
    "A subtask is considered important if it is critical, essential, "
    "or significantly beneficial to achieving the final goal. If it is tangential, "
    "optional, or has minimal impact, then it is not important."
)

eval_definitions = {
    "complexity": complexity_definition,
    "coherence": coherence_definition,
    "importance": importance_definition,
}

goal = "I want to construct a knowledge graph from a dataset of Wikipedia articles."
node = MCT_Node(
    id="1",
    MCT_parent_id="0",
    MCT_id="1",
    label="extract_entities",
    print_label="Extract Entities",  
    task="Extract Entities",
    description="Identify and extract relevant entities from the collection of documents.",
    explanation="This step involves using NLP techniques to identify and extract key entities.",  
    parentIds=["0"],  
)
parent_node = MCT_Node(
    id="0",
    MCT_parent_id=None,
    MCT_id="0",  
    label="root",  
    print_label="Root Node",  
    task="Root Node",
    description="Start of the process.",
    explanation="Initial node for the process.",  
    parentIds=[],  
)

complexity_few_shot_examples = [
    # {
    #     "user_evaluation": 2,
    #     "user_reasoning": "This task requires understanding of entity extraction and knowledge graphs.",
    #     "node": MCT_Node(
    #         id="example_1",
    #         MCT_id="example_1",
    #         label="example_entity_extraction",
    #         print_label="Entity Extraction Example",
    #         task="Extract entities from text using NLP techniques",
    #         description="Extract entities from text using NLP techniques and create relationships between them.",
    #         explanation="This involves using NLP to identify and extract key entities from text.",
    #         parentIds=[],
    #         MCT_parent_id=None
    #     ),
    #     "parent_node": MCT_Node(
    #         id="example_0",
    #         MCT_id="example_0",
    #         label="example_root",
    #         print_label="Root Example",
    #         task="Root Task",
    #         description="Root task description",
    #         explanation="Initial task",
    #         parentIds=[],
    #         MCT_parent_id=None
    #     )
    # },
    {
        "user_evaluation": 1,
        "user_reasoning": "This task is relatively straightforward and uses common NLP techniques.",
        "node": MCT_Node(
            id="example_2", 
            MCT_id="example_2",
            label="example_simple_extraction",
            print_label="Simple Extraction Example",
            task="Extract basic named entities from text",
            description="Use standard NER to identify people, organizations, and locations in text.",
            explanation="Apply basic named entity recognition to extract common entity types.",
            parentIds=[],
            MCT_parent_id=None
        ),
        "parent_node": MCT_Node(
            id="example_0",
            MCT_id="example_0", 
            label="example_root",
            print_label="Root Example",
            task="Root Task",
            description="Root task description",
            explanation="Initial task",
            parentIds=[],
            MCT_parent_id=None
        )
    }
]

eval_few_shot_examples = {
    "complexity": complexity_few_shot_examples,
    "coherence": [],
    "importance": [],
}

eval_params = [(goal, node, parent_node)]
results_grouped, reasons_grouped, num_agents = await criteria.run_all_evaluations(
    goal=goal,
    eval_params=eval_params,
    eval_definitions=eval_definitions,
    eval_few_shot_examples=eval_few_shot_examples,
)

print(f"Number of agents: {num_agents}")
print("\nResults for each node:")
for i, (results, reasons) in enumerate(zip(results_grouped, reasons_grouped)):
    print(f"\nNode {i + 1}:")
    print(f"Complexity score: {results[0]}/{num_agents}")
    print(f"Coherence score: {results[1]}/{num_agents}")
    print(f"Importance score: {results[2]}/{num_agents}")
    print("\nReasons:")
    print(f"Complexity: {reasons[0]}")
    print(f"Coherence: {reasons[1]}")
    print(f"Importance: {reasons[2]}")

Number of agents: 3

Results for each node:

Node 1:
Complexity score: 3/3
Coherence score: 3/3
Importance score: 3/3

Reasons:
Complexity: The task of identifying and extracting entities is a fundamental and straightforward operation in natural language processing. It typically involves the use of Named Entity Recognition (NER) or similar techniques. This type of task does not require advanced knowledge, complex reasoning, or multi-step decision-making. The task employs well-established methods in entity recognition and does not entail any challenging conditions or specialized expertise. It is characterized as a basic, standard procedure within the domain of NLP.
Coherence: The transition from the parent task "Start of the process" to the child task "extract_entities" is logical and coherent. The parent part sets up the initial context, and entity extraction is a well-defined, typical first step in many document processing workflows, making it a natural progression from the initializa